<a href="https://colab.research.google.com/github/AjaharDaroga/BOOK_RECOMMENDATION_SYSTEM/blob/main/BOOK_RECOMMENDATION_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The main objective is to create a book recommendation system for users.**

In [1]:
# importing Basic libraries
import pandas as pd               #DataFrame
import numpy as np                #arrays & matrices
import seaborn as sn              #visualization library
import matplotlib.pyplot as plt   #visualization library
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Importing Data

In [2]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [62]:
Books = pd.read_csv("/content/drive/MyDrive/CAPSTONE_PROJECTS/UNSUPERVISED_ML_PROJECT/DATA/Books.csv")
Users = pd.read_csv("/content/drive/MyDrive/CAPSTONE_PROJECTS/UNSUPERVISED_ML_PROJECT/DATA/Users.csv")
Ratings = pd.read_csv("/content/drive/MyDrive/CAPSTONE_PROJECTS/UNSUPERVISED_ML_PROJECT/DATA/Ratings.csv")

# DATASETS

In [63]:
# view the data
Books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [64]:
Users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [65]:
Ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


# DATA PREPARATION FOR BOOKS DATASET

In [66]:
# shape of dataset (i.e row & col)
Books.shape,Users.shape,Ratings.shape

((271360, 8), (278858, 3), (1149780, 3))

Understand the Books dataset

In [67]:
# info of dataset
Books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [68]:
# missing data
Books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [69]:
# Rename the feature name for performing operation
Books.columns = ['ISBN', 'Book_Title', 'Book_Author', 'Year_Of_Publication', 'Publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
Books.head()

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [70]:
# Feaching the missing values
Books.loc[Books.Book_Author.isnull(),:]

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher,imageUrlS,imageUrlM,imageUrlL
187689,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...


In [71]:
Books.loc[Books.Publisher.isnull(),:]

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher,imageUrlS,imageUrlM,imageUrlL
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...
129037,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...


In [72]:
# imputing the missing values
Books.loc[(Books.ISBN == '9627982032'), 'Book_Author'] = 'No_Author'
Books.loc[(Books.ISBN == '193169656X'), 'Publisher'] = 'No_Publisher'
Books.loc[(Books.ISBN == '1931696993'), 'Publisher'] = 'No_Publisher'

In [73]:
# checking missing data after 
Books.isnull().sum()

ISBN                   0
Book_Title             0
Book_Author            0
Year_Of_Publication    0
Publisher              0
imageUrlS              0
imageUrlM              0
imageUrlL              3
dtype: int64

In [74]:
# Removing the images from dataset beacuse its not required for model.
Books.drop(['imageUrlS', 'imageUrlM', 'imageUrlL'], axis=1, inplace=True)

In [75]:
# unique 
Books.nunique(axis=0)

ISBN                   271360
Book_Title             242135
Book_Author            102024
Year_Of_Publication       202
Publisher               16808
dtype: int64

In [76]:
# Checking for unique values because data type is object
Books.Year_Of_Publication.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

"0" , 'DK Publishing Inc','Gallimard' & year greater than 2022 is the wrong data entry in year of publication feature.

In [77]:
Books.loc[Books.Year_Of_Publication == 'DK Publishing Inc', :]

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


In [78]:
Books.loc[Books.Year_Of_Publication == 'Gallimard', :]

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...


After seen above recoreds we understand the mispalce the info in columns

In [79]:
Books.at[209538 ,'Book_Title'] = 'DK Readers: Creating the X-Men, How It All Begen (Level 4: Proficient Readers)'
Books.at[209538 ,'Book_Author'] = 'Michael Teitelbaum'
Books.at[209538 ,'Year_Of_Publication'] = 2000
Books.at[209538 ,'Publisher'] = 'DK Publishing Inc'

Books.at[221678 ,'Book_Title'] = 'DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)'
Books.at[221678 ,'Book_Author'] = 'James Buckley'
Books.at[221678 ,'Year_Of_Publication'] = 2000
Books.at[221678 ,'Publisher'] = 'DK Publishing Inc'

Books.at[220731 ,'Book_Title'] = 'Peuple du ciel, suivi de Les Bergers'
Books.at[220731 ,'Book_Author'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'
Books.at[220731 ,'Year_Of_Publication'] = 2003
Books.at[220731 ,'Publisher'] = 'Gallimard'

In [81]:
Books.Year_Of_Publication = pd.to_numeric(Books.Year_Of_Publication, errors = 'coerce')

In [84]:
# imputing the wrong entry with mean()
Books.loc[(Books.Year_Of_Publication > 2022) | (Books.Year_Of_Publication == 0), 'Year_Of_Publication'] = np.NAN
Books.Year_Of_Publication.fillna(round(Books.Year_Of_Publication.mean()), inplace = True)

In [85]:
# Converting year of publication in Numbers
Books.Year_Of_Publication = Books.Year_Of_Publication.astype(np.int32)

In [86]:
# Final Dataset for Books
df_book=Books.copy()
df_book.head()

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


# DATA PREPRATION FOR USERS DATASET

In [87]:
# # view the data
Users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


Location Feature contains CITY, STATE, COUNTRY & Age feature has missing values

In [88]:
# shape of dataset (i.e row & col)
Users.shape

(278858, 3)

In [89]:
# info of dataset
Users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [90]:
# missing data
Users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [92]:
# checking for unique value so that identify wrong entry
print(sorted(list(Users['Age'].unique())))

[nan, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 113.0, 114.0, 115.0, 116.0, 118.0, 119.0, 123.0, 124.0, 127.0, 128.0, 132.0, 133.0, 136.0, 137.0, 138.0, 140.0, 141.0, 143.0, 146.0, 147.0, 148.0, 151.0, 152.0, 156.0, 157.0, 159.0, 162.0, 168.0, 172.0, 175.0, 183.0, 186.0, 189.0, 199.0, 200.0, 201.0, 204.0, 207.0, 208.0, 209.0, 210.0, 212.0, 219.0, 220.0, 223.0, 226.0

In [94]:
# Imputing outlier with mean values
Users.loc[(Users.Age > 90) | (Users.Age < 10), 'Age'] = np.nan
Users.Age = Users.Age.fillna(Users.Age.mean())
Users.Age = Users.Age.astype(np.int32) #changing Datatype to int
# for checking
print(sorted(Users.Age.unique()))

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]
